In [1]:
import h5py
import tensorflow as tf
import keras
import dask.array as da
#Check if GPU is being used and import other necessary libraries
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
#Again, just import some necessary libraries as part of the preamble in training our dataset
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.initializers import TruncatedNormal, Zeros
from keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
#Just define some variables for the training set and test set
f = h5py.File("train_pile_10000000.h5", "r")
g = h5py.File("test_pile_5000000.h5", "r")

In [4]:
#Print a dictionary to see how the dataset looks like
print(list(f.keys()))
print(list(g.keys()))

['features', 'targets']
['features', 'targets']


In [5]:
X_train = da.from_array(f['features'], chunks=(200000, 1, 32,32))
y_train =  da.from_array(f['targets'], chunks=(200000,1))
X_test = da.from_array(g['features'], chunks=(200000, 1, 32,32))
y_test =  da.from_array(g['targets'], chunks=(200000,1))

In [6]:
X_val = X_train[8000000:]
y_val = y_train[8000000:]
X_train = X_train[:8000000]
y_train = y_train[:8000000]

In [7]:
X_train.shape, y_train.shape, X_val.shape,  y_val.shape, X_test.shape, y_test.shape

((8000000, 1, 32, 32),
 (8000000, 1),
 (2000000, 1, 32, 32),
 (2000000, 1),
 (5000000, 1, 32, 32),
 (5000000, 1))

In [8]:
def dask_generator(X, y, batch_size):
    instances = X.shape[0]
    batches = instances // batch_size
    while True:
        for i in range(batches):
            X_batch = X[i*batch_size:(i+1)*batch_size]
            y_batch = y[i*batch_size:(i+1)*batch_size]
            yield X_batch, y_batch

In [9]:
batch_size = 64
train = dask_generator(X_train, y_train, batch_size)
test = dask_generator(X_test, y_test, batch_size)
val = dask_generator(X_val, y_val, batch_size)

In [ ]:
###############################################################################################################################

In [ ]:
###############################################################################################################################

In [ ]:
#Train for Traditional Conv2D

In [10]:
inputs1 = Input(shape=(1,32,32), batch_size = 64)
x1 = Conv2D(32, kernel_size=2,  padding='same', strides = 1, activation='relu', kernel_initializer='TruncatedNormal',
           data_format='channels_first')(inputs1)
x1 = Conv2D(32,  kernel_size=2, padding='same', strides = 1, activation='relu',kernel_initializer='TruncatedNormal',
           data_format='channels_first')(x1)
x1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(x1)
x1 = Conv2D(64,  kernel_size=3, padding='same', strides = 1, activation='relu',kernel_initializer='TruncatedNormal',
           data_format='channels_first')(x1)
x1 = Conv2D(64,  kernel_size=3, padding='same', strides = 1, activation='relu',kernel_initializer='TruncatedNormal',
           data_format='channels_first')(x1)
x1 = Flatten()(x1)
x1 = Dense(128, activation='relu', kernel_initializer='TruncatedNormal')(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(32, activation='relu', kernel_initializer='TruncatedNormal')(x1)
x1 = Dropout(0.2)(x1)
outputs1 = Dense(1, activation='sigmoid', kernel_initializer='TruncatedNormal')(x1)
conv2d = keras.Model(inputs=inputs1, outputs=outputs1)

In [13]:
conv2d.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1.e-6)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=5, restore_best_weights=True)
callbacks = [lr_scheduler, early_stopping_cb]

In [14]:
conv2d.fit(train, steps_per_epoch = X_train.shape[0]//64, validation_data = val,
           validation_steps = X_val.shape[0]//64, shuffle = True, epochs=100, callbacks = callbacks)

Epoch 1/100
125000/125000 [==============================] - 805s 6ms/step - loss: 0.3242 - accuracy: 0.8710 - val_loss: 0.3139 - val_accuracy: 0.8742 - lr: 0.0010
Epoch 2/100
125000/125000 [==============================] - 800s 6ms/step - loss: 0.3133 - accuracy: 0.8762 - val_loss: 0.3033 - val_accuracy: 0.8779 - lr: 0.0010
Epoch 3/100
125000/125000 [==============================] - 799s 6ms/step - loss: 0.3118 - accuracy: 0.8769 - val_loss: 0.3050 - val_accuracy: 0.8773 - lr: 0.0010
Epoch 4/100
125000/125000 [==============================] - 799s 6ms/step - loss: 0.3115 - accuracy: 0.8771 - val_loss: 0.3045 - val_accuracy: 0.8781 - lr: 0.0010
Epoch 5/100
125000/125000 [==============================] - 797s 6ms/step - loss: 0.3116 - accuracy: 0.8772 - val_loss: 0.3046 - val_accuracy: 0.8782 - lr: 0.0010
Epoch 6/100
125000/125000 [==============================] - 796s 6ms/step - loss: 0.3010 - accuracy: 0.8811 - val_loss: 0.2973 - val_accuracy: 0.8812 - lr: 1.0000e-04
Epoch 7/100


In [ ]:
###############################################################################################################################

In [ ]:
###############################################################################################################################

In [ ]:
#For F-FCNN now

In [10]:
from DeepSaki.layers import FourierConvolution2D #Import this library for the training

C:\Users\Sham\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [24]:
inputs = keras.Input(shape=(1,32,32), batch_size = 64)
x = FourierConvolution2D(filters = 32, kernels = (3,3), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(inputs)
x = FourierConvolution2D(filters = 32, kernels = (3,3), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(x)
x = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(x)
x = FourierConvolution2D(filters = 64, kernels = (2,2), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(x)
x = FourierConvolution2D(filters = 64, kernels = (2,2), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(x)
x= Flatten()(x)
x = Dense(128, activation='relu', kernel_initializer='TruncatedNormal')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu', kernel_initializer='TruncatedNormal')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer='TruncatedNormal')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [25]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [27]:
model.fit(train, steps_per_epoch = X_train.shape[0]//64, validation_data = val,
           validation_steps = X_val.shape[0]//64, shuffle = True, epochs=100, callbacks = callbacks)

Epoch 1/100
125000/125000 [==============================] - 3669s 29ms/step - loss: 0.3443 - accuracy: 0.8606 - val_loss: 0.3363 - val_accuracy: 0.8653 - lr: 0.0010
Epoch 2/100
125000/125000 [==============================] - 3659s 29ms/step - loss: 0.3409 - accuracy: 0.8624 - val_loss: 0.3345 - val_accuracy: 0.8663 - lr: 0.0010
Epoch 3/100
125000/125000 [==============================] - 3660s 29ms/step - loss: 0.3399 - accuracy: 0.8630 - val_loss: 0.3328 - val_accuracy: 0.8663 - lr: 0.0010
Epoch 4/100
125000/125000 [==============================] - 3667s 29ms/step - loss: 0.3394 - accuracy: 0.8631 - val_loss: 0.3304 - val_accuracy: 0.8676 - lr: 0.0010
Epoch 5/100
125000/125000 [==============================] - 3674s 29ms/step - loss: 0.3391 - accuracy: 0.8634 - val_loss: 0.3321 - val_accuracy: 0.8677 - lr: 0.0010
Epoch 6/100
125000/125000 [==============================] - 3673s 29ms/step - loss: 0.3390 - accuracy: 0.8636 - val_loss: 0.3334 - val_accuracy: 0.8671 - lr: 0.0010
Epoc

In [ ]:
###############################################################################################################################

In [ ]:
###############################################################################################################################

In [ ]:
#Linearly stacked model conv2d and ffcnn

In [18]:
inputs2 = Input(shape=(1,32,32), batch_size = 64)
x2 = Conv2D(32, kernel_size=2,  padding='same', strides = 1, activation='relu', kernel_initializer='TruncatedNormal',
           data_format='channels_first')(inputs2)
x2 = Conv2D(32,  kernel_size=2, padding='same', strides = 1, activation='relu',kernel_initializer='TruncatedNormal',
           data_format='channels_first')(x2)
x2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(x2)
x2 = FourierConvolution2D(filters = 64, kernels = (2,2), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(x2)
x2 = FourierConvolution2D(filters = 64, kernels = (2,2), kernel_initializer = tf.keras.initializers.TruncatedNormal(),
                        use_bias=False, isChannelFirst = True, padToPower2 = True, applyConjugate = False,
                        method = "MATRIX_PRODUCT")(x2)
x2 = Flatten()(x2)
x2 = Dense(128, activation='relu', kernel_initializer='TruncatedNormal')(x2)
x2 = Dropout(0.2)(x2)
x2 = Dense(32, activation='relu', kernel_initializer='TruncatedNormal')(x2)
x2 = Dropout(0.2)(x2)
outputs2 = Dense(1, activation='sigmoid', kernel_initializer='TruncatedNormal')(x2)
stack = keras.Model(inputs=inputs2, outputs=outputs2)

In [19]:
stack.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [20]:
stack.fit(train, steps_per_epoch = X_train.shape[0]//64, validation_data = val,
           validation_steps = X_val.shape[0]//64, shuffle = True, epochs=100, callbacks = callbacks)

Epoch 1/100
125000/125000 [==============================] - 1859s 15ms/step - loss: 0.3313 - accuracy: 0.8680 - val_loss: 0.3202 - val_accuracy: 0.8698 - lr: 0.0010
Epoch 2/100
125000/125000 [==============================] - 1854s 15ms/step - loss: 0.3216 - accuracy: 0.8727 - val_loss: 0.3173 - val_accuracy: 0.8719 - lr: 0.0010
Epoch 3/100
125000/125000 [==============================] - 1856s 15ms/step - loss: 0.3200 - accuracy: 0.8734 - val_loss: 0.3263 - val_accuracy: 0.8654 - lr: 0.0010
Epoch 4/100
125000/125000 [==============================] - 1855s 15ms/step - loss: 0.3191 - accuracy: 0.8740 - val_loss: 0.3112 - val_accuracy: 0.8763 - lr: 0.0010
Epoch 5/100
125000/125000 [==============================] - 1855s 15ms/step - loss: 0.3186 - accuracy: 0.8743 - val_loss: 0.3109 - val_accuracy: 0.8746 - lr: 0.0010
Epoch 6/100
125000/125000 [==============================] - 1856s 15ms/step - loss: 0.3182 - accuracy: 0.8746 - val_loss: 0.3148 - val_accuracy: 0.8735 - lr: 0.0010
Epoc

In [ ]:
########################################################################################################################

In [ ]:
########################################################################################################################

In [ ]:
#Save the model for evaluation in the test set later on

In [ ]:
model.save("ffcnnmodel.h5")
conv2d.save("conv2dmodel.h5")
stack.save("stackmodel.h5")